In [1]:
# Librerias

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go

In [2]:
# Leer rutas

from pathlib import Path

PROJECT_ROOT = Path.cwd().parents[1]
DATA_PATH = PROJECT_ROOT / "data"

print("PROJECT_ROOT:", PROJECT_ROOT)
print("DATA_PATH:", DATA_PATH)
print("Existe data:", DATA_PATH.exists())

PROJECT_ROOT: c:\Users\HP Spectre X360\Desktop\MARIANA\COLLOQUIA_2025\Analisis_Powermeter
DATA_PATH: c:\Users\HP Spectre X360\Desktop\MARIANA\COLLOQUIA_2025\Analisis_Powermeter\data
Existe data: True


In [3]:
# Parametros de seguridad
umbral_pico = 0.5
umbral_desequilibrio = 0.1

max_i_r_ple1 = 0.01
max_i_s_ple1 = 0.01
max_i_t_ple1 = 0.01

max_i_r_ple7 = 0.01
max_i_s_ple7 = 0.01
max_i_t_ple7 = 0.01

tiempo_min = pd.Timedelta(minutes=5)
cant_min_picos = 10
tiempo_min_picos = pd.Timedelta(minutes=60)



In [4]:
# Diccionario de parametros de seguridad

limites_corriente = {
    'PLE1': {
        'R': max_i_r_ple1,
        'S': max_i_s_ple1,
        'T': max_i_t_ple1,
    },
    'PLE7': {
        'R': max_i_r_ple7,
        'S': max_i_s_ple7,
        'T': max_i_t_ple7,
    }
}

In [5]:
# Funcion para cargar datos

from glob import glob
import pandas as pd

def cargar_maquina(base_path, maquina):
    paths = glob(f"{base_path}/{maquina}/**/*.csv", recursive=True)

    dfs = []
    for path in paths:
        df = pd.read_csv(path)
        df['maquina'] = maquina
        dfs.append(df)

    return (
        pd.concat(dfs, ignore_index=True)
          .sort_values('temporal_placa')
          .reset_index(drop=True)
    )


In [6]:
# Carga de datos

df_ple1 = cargar_maquina(DATA_PATH, "PLE1")
df_ple7 = cargar_maquina(DATA_PATH, "PLE7")

In [7]:
# Funcion preparar_df

def preparar_df(df):
    df = df.copy()

    # Timestamp
    df['temporal_placa'] = pd.to_datetime(df['temporal_placa'])
    df['hora'] = df['temporal_placa'].dt.hour
    df['minuto'] = df['temporal_placa'].dt.minute

    # Turnos
    def asignar_turno(hora, minuto):
        t = hora * 60 + minuto

        # Pausas
        if 12*60 <= t < 12*60 + 30:
            return 'ALMUERZO'
        if 22*60 <= t < 22*60 + 30:
            return 'CENA'

        # Turnos
        if 5*60 <= t < 17*60:
            return 'TURNO MAÑANA'
        if 17*60 <= t < 22*60:
            return 'TURNO TARDE'
        if (t >= 22*60 + 30) or (t < 1*60):
            return 'TURNO TARDE'

        return 'FUERA_TURNO'

    df['turno'] = df.apply(
        lambda x: asignar_turno(x['hora'], x['minuto']),
        axis=1
    )
        
    # Potencias totales por timestamp
    df['p_activa_total'] = (
        df['potencia_a_r'] +
        df['potencia_a_s'] +
        df['potencia_a_t']
    )

    df['q_reactiva_total'] = (
        df['potencia_r_r'] +
        df['potencia_r_s'] +
        df['potencia_r_t']
    )

    return df


In [8]:
# Ajuste de datos de las plegadoras

df_ple1 = preparar_df(df_ple1)
df_ple7 = preparar_df(df_ple7)

df_all = pd.concat([df_ple1, df_ple7], ignore_index=True)

# Filtra los datos correspondientes al 9-1-2026
import datetime as dt

dia = dt.date(2026, 1, 9)
df_all = df_all[df_all['temporal_placa'].dt.date == dia]

In [9]:
# Calculo la media de las corrientes excluyendo los picos

def media_sin_picos(s, q = 0.9):
    return s[s <= s.quantile(q)].mean()

medias_por_fase = (
    df_all
    .groupby('maquina')
    .agg(
        i_r_media=('corriente_r', media_sin_picos),
        i_s_media=('corriente_s', media_sin_picos),
        i_t_media=('corriente_t', media_sin_picos),
    )
)

medias_por_fase

,i_r_media,i_s_media,i_t_media
maquina,,,
PLE1,16.568281,17.697971,16.546234
PLE7,11.917244,25.300470,28.839551


In [10]:
import numpy as np
from scipy.signal import cheby2, filtfilt

def cheby2_lowpass(signal, fs, fc, order=4, rs=30):
    """
    Filtro Chebyshev tipo II pasa-bajos sin desfase.
    """
    wn = fc / (fs / 2)  # normalizado a Nyquist
    b, a = cheby2(order, rs, wn, btype='low')
    return filtfilt(b, a, signal)


In [11]:
fs = 0.5     # Hz  ← AJUSTÁ según tu muestreo real
fc = 0.015   # Hz  (~60–70 s) elimina picos rápidos

df_filt = []

for maq, g in df_all.groupby('maquina'):

    g = g.sort_values('temporal_placa').copy()

    # Protección mínima
    if len(g) < 10:
        continue

    for fase in ['r', 's', 't']:
        col = f'corriente_{fase}'
        col_f = f'corriente_{fase}_filt'

        g[col_f] = cheby2_lowpass(
            g[col].values,
            fs=fs,
            fc=fc
        )

    df_filt.append(g)

df_all_filt = (
    np.concatenate(df_filt)
    if isinstance(df_filt[0], np.ndarray)
    else pd.concat(df_filt, ignore_index=True)
)

medias_filtradas = (
    df_all_filt
    .groupby('maquina')[[
        'corriente_r_filt',
        'corriente_s_filt',
        'corriente_t_filt'
    ]]
    .mean()
)

medias_filtradas

,corriente_r_filt,corriente_s_filt,corriente_t_filt
maquina,,,
PLE1,16.798920,17.941149,16.801057
PLE7,12.088017,25.663479,29.213956


In [12]:
# Funcion detectar_picos

def detectar_picos(signal, timestamp, umbral_pico):
    dt = timestamp.diff().dt.total_seconds()
    valid = dt > 0

    dI_dt = signal.diff() / dt

    return (
        valid &
        dI_dt.notna() &
        (dI_dt > umbral_pico)
    )

In [13]:
# Detección de picos en las 3 fases

import pandas as pd

maquinas = df_all['maquina'].unique()

fase_cols = {
    'R': 'corriente_r',
    'S': 'corriente_s',
    'T': 'corriente_t'
}

df_peaks_list = []

for maq in maquinas:

    g = (
        df_all[df_all['maquina'] == maq]
        .sort_values('temporal_placa')
        .copy()
    )

    if g.empty:
        continue

    for fase, col_corriente in fase_cols.items():

        señal = g[col_corriente]
        timestamp = g['temporal_placa']

        idx_picos = detectar_picos(señal, timestamp, umbral_pico)

        if len(idx_picos) == 0:
            continue

        df_peaks = g.loc[idx_picos, ['temporal_placa']].copy()
        df_peaks['maquina'] = maq
        df_peaks['fase'] = fase    

        df_peaks_list.append(df_peaks)

# DataFrame final de picos
df_peaks_all = pd.concat(df_peaks_list, ignore_index=True)

df_peaks_all['temporal_placa'] = (
    pd.to_datetime(df_peaks_all['temporal_placa'], errors='coerce')
    .dt.tz_localize(None)
)


In [14]:
# Visualizacion de picos detectados y su marca temporal

df_peaks_all = df_peaks_all.sort_values(
    by=['maquina', 'fase']
).reset_index(drop=True)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

df_peaks_all

,temporal_placa,maquina,fase
0,2026-01-09 16:57:16,PLE1,R
1,2026-01-09 16:57:37,PLE1,R
2,2026-01-09 16:57:49,PLE1,R
3,2026-01-09 16:58:04,PLE1,R
4,2026-01-09 17:04:53,PLE1,R
5,2026-01-09 17:05:03,PLE1,R
6,2026-01-09 17:05:41,PLE1,R
7,2026-01-09 17:11:22,PLE1,R
8,2026-01-09 17:11:27,PLE1,R
9,2026-01-09 17:11:45,PLE1,R


In [15]:
# Filtrado de picos peligrosos + timestamp para alarmas

peaks_rows = []

for maq, g in df_all.groupby('maquina'):

    g = g.sort_values('temporal_placa').reset_index(drop=True)

    for fase, col in {'r': 'corriente_r',
                      's': 'corriente_s',
                      't': 'corriente_t'}.items():

        media_ref = medias_por_fase.loc[maq, f'i_{fase}_media']

        # 1) detectar flancos por derivada
        mask_picos = detectar_picos(
            g[col],
            g['temporal_placa'],
            umbral_pico
        )

        # 2) filtrar por amplitud: 3 × media base
        mask_grande = g[col] >= 2 * media_ref

        # 3) pico válido = derivada fuerte + amplitud grande
        mask_final = mask_picos & mask_grande

        p = g[mask_final]

        for _, r in p.iterrows():
            peaks_rows.append({
                'maquina': maq,
                'fase': fase,
                'temporal_placa': r['temporal_placa'],
                'valor_corriente': r[col],
                'media_base': media_ref
            })

df_peaks_alarmas = pd.DataFrame(peaks_rows)


In [16]:
# Alarmas

import pandas as pd

alarmas = []

# ======================================================
# PREPROCESO GLOBAL (desequilibrio)
# ======================================================
df = df_all.copy()

df['i_avg'] = df[['corriente_r','corriente_s','corriente_t']].mean(axis=1)

df['desequilibrio_pct'] = (
    df[['corriente_r','corriente_s','corriente_t']]
    .sub(df['i_avg'], axis=0)
    .abs()
    .max(axis=1)
    / df['i_avg']
)

df['desequilibrio'] = df['desequilibrio_pct'] >= umbral_desequilibrio
# ======================================================
# PROCESO POR MÁQUINA
# ======================================================
for maq, g in df.groupby('maquina'):

    g = g.sort_values('temporal_placa')

    if maq not in limites_corriente or maq not in medias_por_fase.index:
        continue

    # --- Medias por fase ---
    media = {
        'R': medias_por_fase.loc[maq, 'i_r_media'],
        'S': medias_por_fase.loc[maq, 'i_s_media'],
        'T': medias_por_fase.loc[maq, 'i_t_media'],
    }

    # --- Límites absolutos ---
    limite_abs = {
        f: media[f] * (1 + limites_corriente[maq][f])
        for f in ['R','S','T']
    }

    # --- Flags de sobrecorriente ---
    g['exceso_R'] = g['corriente_r'] > limite_abs['R']
    g['exceso_S'] = g['corriente_s'] > limite_abs['S']
    g['exceso_T'] = g['corriente_t'] > limite_abs['T']
    g['exceso_RST'] = g['exceso_R'] & g['exceso_S'] & g['exceso_T']

    # ==================================================
    # A) SOBRECORRIENTE TRIFÁSICA
    # ==================================================
    g['bloque_rst'] = (g['exceso_RST'] != g['exceso_RST'].shift()).cumsum()

    for _, b in g[g['exceso_RST']].groupby('bloque_rst'):

        duracion = b['temporal_placa'].iloc[-1] - b['temporal_placa'].iloc[0]

        if duracion >= tiempo_min:

            medias_fases = {
                'R': b['corriente_r'].mean(),
                'S': b['corriente_s'].mean(),
                'T': b['corriente_t'].mean(),
            }

            fase_critica = max(medias_fases, key=medias_fases.get)

            alarmas.append({
                'maquina': maq,
                'tipo_alarma': 'SOBRECORRIENTE_TRIFASICA',
                'descripcion': 'Sobrecorriente sostenida en las tres fases',
                'ubicacion': 'RST',
                'criterio': 'MAX_FASE',
                'fase_critica': fase_critica,
                'valor_medido': medias_fases[fase_critica],
                'valor_limite': limite_abs[fase_critica],
                'fecha_primera_deteccion': b['temporal_placa'].iloc[0],
                'duracion_min': duracion.total_seconds() / 60
            })

    # ==================================================
    # B) SOBRECORRIENTE MONOFÁSICA
    # ==================================================
    for fase, col in {'R':'corriente_r','S':'corriente_s','T':'corriente_t'}.items():

        flag = f'exceso_{fase}'
        g_fase = g[g[flag] & ~g['exceso_RST']].copy()

        if g_fase.empty:
            continue

        g_fase['bloque'] = (g_fase[flag] != g_fase[flag].shift()).cumsum()

        for _, b in g_fase.groupby('bloque'):

            duracion = b['temporal_placa'].iloc[-1] - b['temporal_placa'].iloc[0]

            if duracion >= tiempo_min:
                alarmas.append({
                    'maquina': maq,
                    'tipo_alarma': 'SOBRECORRIENTE',
                    'descripcion': 'Sobrecorriente sostenida monofásica',
                    'ubicacion': fase,
                    'criterio': 'FASE_UNICA',
                    'fase_critica':fase,
                    'valor_medido': b[col].mean(),
                    'valor_limite': limite_abs[fase],
                    'fecha_primera_deteccion': b['temporal_placa'].iloc[0],
                    'duracion_min': duracion.total_seconds() / 60
                })

    # ==================================================
    # C) DESEQUILIBRIO TRIFÁSICO
    # ==================================================
    g['bloque_des'] = (g['desequilibrio'] != g['desequilibrio'].shift()).cumsum()

    for _, b in g[g['desequilibrio']].groupby('bloque_des'):

        duracion = b['temporal_placa'].iloc[-1] - b['temporal_placa'].iloc[0]

        if duracion >= tiempo_min:
            alarmas.append({
                'maquina': maq,
                'tipo_alarma': 'DESEQUILIBRIO_TRIFASICO',
                'descripcion': 'Desequilibrio de corrientes trifásicas',
                'ubicacion': 'RST',
                'criterio': 'MAX_DESVIO_SOBRE_MEDIA',
                'fase_critica': fase,
                'valor_medido': b['desequilibrio_pct'].mean() * 100,
                'valor_limite': umbral_desequilibrio * 100,
                'fecha_primera_deteccion': b['temporal_placa'].iloc[0],
                'duracion_min': duracion.total_seconds() / 60
            })

# ======================================================
# D) FRECUENCIA ANÓMALA DE PICOS SEVEROS
# ======================================================

df_picos = (
    df_peaks_alarmas
    .sort_values('temporal_placa')
    .drop_duplicates(subset=['maquina','fase','temporal_placa'])
)

for (maq, fase), gp in df_picos.groupby(['maquina','fase']):

    gp = gp.sort_values('temporal_placa').set_index('temporal_placa')

    # Serie dummy para conteo rolling
    s = pd.Series(1, index=gp.index)

    # Conteo en ventana móvil
    cnt = s.rolling(tiempo_min_picos).sum()

    viol = cnt[cnt >= cant_min_picos]

    if viol.empty:
        continue

    # Primera detección
    t_fin = viol.index[0]
    t_ini = t_fin - tiempo_min_picos

    alarmas.append({
        'maquina': maq,
        'tipo_alarma': 'FRECUENCIA_PICOS',
        'descripcion': 'Frecuencia anómala de picos severos de corriente',
        'ubicacion': fase.upper(),
        'criterio': 'FASE_UNICA',
        'fase_critica': fase,
        'valor_medido': int(cnt.max()),
        'valor_limite': cant_min_picos,
        'fecha_primera_deteccion': t_ini,
        'duracion_min': tiempo_min_picos.total_seconds() / 60
    })

df_alarmas = pd.DataFrame(alarmas)

df_alarmas = df_alarmas.sort_values(
    by=['maquina', 'tipo_alarma']
).reset_index(drop=True)

df_alarmas


,maquina,tipo_alarma,descripcion,ubicacion,criterio,fase_critica,valor_medido,valor_limite,fecha_primera_deteccion,duracion_min
0,PLE1,SOBRECORRIENTE,Sobrecorriente sostenida monofásica,R,FASE_UNICA,R,16.910965,16.733963,2026-01-09 16:56:16-03:00,423.616667
1,PLE1,SOBRECORRIENTE,Sobrecorriente sostenida monofásica,S,FASE_UNICA,S,18.014107,17.874951,2026-01-09 16:56:01-03:00,423.866667
2,PLE1,SOBRECORRIENTE,Sobrecorriente sostenida monofásica,T,FASE_UNICA,T,16.855718,16.711696,2026-01-09 16:58:07-03:00,294.850000
3,PLE1,SOBRECORRIENTE_TRIFASICA,Sobrecorriente sostenida en las tres fases,RST,MAX_FASE,S,18.507284,17.874951,2026-01-09 18:20:31-03:00,7.700000
4,PLE1,SOBRECORRIENTE_TRIFASICA,Sobrecorriente sostenida en las tres fases,RST,MAX_FASE,S,19.094125,17.874951,2026-01-09 18:40:51-03:00,15.983333
5,PLE7,DESEQUILIBRIO_TRIFASICO,Desequilibrio de corrientes trifásicas,RST,MAX_DESVIO_SOBRE_MEDIA,T,45.845613,10.000000,2026-01-09 16:56:03-03:00,423.916667
6,PLE7,SOBRECORRIENTE,Sobrecorriente sostenida monofásica,R,FASE_UNICA,R,12.185444,12.036417,2026-01-09 16:57:31-03:00,323.216667
7,PLE7,SOBRECORRIENTE,Sobrecorriente sostenida monofásica,S,FASE_UNICA,S,25.999154,25.553475,2026-01-09 16:56:31-03:00,423.450000
8,PLE7,SOBRECORRIENTE,Sobrecorriente sostenida monofásica,T,FASE_UNICA,T,29.421900,29.127947,2026-01-09 16:56:31-03:00,423.450000
9,PLE7,SOBRECORRIENTE_TRIFASICA,Sobrecorriente sostenida en las tres fases,RST,MAX_FASE,T,30.044649,29.127947,2026-01-09 18:40:55-03:00,9.983333


In [17]:
# Alrmas a .cvs
# Eventos a .cvs
from pathlib import Path

OUT_TABLAS = PROJECT_ROOT / "output" / "tablas"
OUT_TABLAS.mkdir(parents=True, exist_ok=True)

df_alarmas.to_csv(OUT_TABLAS / "df_alarmas.csv", index=False)
